In [1]:
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from utils import *
from tgp_research_quantitative_tools.optionpricing.ClosedForm.Black import SpreadOptionTwoAssets

###  First, we set the details of our HRCO

In [2]:
as_of = dt.datetime(2024, 2, 10) # date of reference fwd curve

start_simulation_date = dt.datetime(2024, 5, 31)
end_simulation_date = dt.datetime(2025, 7, 1)

start_contract_date = dt.datetime(2024, 6, 1)
end_contract_date = dt.datetime(2025, 6, 30)

power_market = "ERCOTHOUHUB"
gas_market = "HH_gas"

granularity = "daily"
methodology = "spot" # we use spot simulations since we need the 1-day ahead prices

heat_rate = 12 # MMBtu/MWh
gas_adder = 0.05 # $/MMBtu
start_charge = 700 # MMBtu (to be multiplied by the gas price)
VOM = 0.8 # $/MWh
daily_volume = 24 * 200 # MWh

energy_payment_delay = 20 # days from delivery to payment date
payment_grouped_monthly = True # True if Energy Payments are group by month, False otherwise 

# Note that we simplify the pricing by assuming that the buyer of the HRCO only books slots of 24h each day.
# This follows the reasoning that if the MtM is positive, he will make the most out of it by booking the maximum daily amount available.

nb_scenarios = 300
interest_rate = 0.0425

### Simulate spot prices for gas and power

In [3]:
simulation = simulate_fwd_prices(
    [power_market, gas_market], 
    start_simulation_date,
    end_simulation_date, 
    methodology,
    granularity, 
    as_of=as_of, 
    start_fwd_date=start_contract_date,
    nb_simulation=nb_scenarios,
)

c:\Users\AMIREL\Desktop\Research HRCO\venv\lib\site-packages\tgp_research_quantitative_tools\optionpricing\MonteCarlo\two_factor_model.py:96: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_temp2 = pd.concat([df_temp, df_temp.shift(1)], axis=1).fillna(method='bfill')
c:\Users\AMIREL\Desktop\Research HRCO\venv\lib\site-packages\tgp_research_quantitative_tools\optionpricing\MonteCarlo\two_factor_model.py:96: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_temp2 = pd.concat([df_temp, df_temp.shift(1)], axis=1).fillna(method='bfill')
c:\Users\AMIREL\Desktop\Research HRCO\venv\lib\site-packages\tgp_research_quantitative_tools\optionpricing\MonteCarlo\two_factor_model.py:96: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.


Time for solve_minimization_problem : 0.09 seconds


c:\Users\AMIREL\Desktop\Research HRCO\venv\lib\site-packages\tgp_research_quantitative_tools\optionpricing\MonteCarlo\two_factor_model.py:96: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_temp2 = pd.concat([df_temp, df_temp.shift(1)], axis=1).fillna(method='bfill')
c:\Users\AMIREL\Desktop\Research HRCO\venv\lib\site-packages\tgp_research_quantitative_tools\optionpricing\MonteCarlo\two_factor_model.py:96: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_temp2 = pd.concat([df_temp, df_temp.shift(1)], axis=1).fillna(method='bfill')
c:\Users\AMIREL\Desktop\Research HRCO\venv\lib\site-packages\tgp_research_quantitative_tools\optionpricing\MonteCarlo\two_factor_model.py:96: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.


Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.301864738880471e-05
            Iterations: 9
            Function evaluations: 49
            Gradient evaluations: 9
Time for solve_minimization_problem : 0.06 seconds
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.003856476561620994
            Iterations: 52
            Function evaluations: 1040
            Gradient evaluations: 52
Time for solve_minimization_problem : 0.13 seconds


c:\Users\AMIREL\Desktop\Research HRCO\venv\lib\site-packages\tgp_research_quantitative_tools\optionpricing\MonteCarlo\DiffusionProcess.py:51: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  self.hourly_index_date_range = pd.date_range(self.start_date, self.end_date + dt.timedelta(days=1),


Time for compute_daily_gas_shaping_from_monthly_assessments : 0.03 seconds


### HRCO Pricing

Build some dfs that we need

In [4]:
simulation_dates = pd.date_range(start=start_simulation_date, end=end_simulation_date, freq='D')

power_prices_df = pd.DataFrame(simulation[power_market].values, index=simulation_dates)
gas_prices_df = pd.DataFrame(simulation[gas_market].values, index=simulation_dates)

In [5]:
gas_prices_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
2024-05-31,2.000231,2.136700,2.132431,2.088117,1.974396,2.050867,2.088117,2.098583,2.054973,2.028431,...,2.153862,2.006241,2.140978,2.130300,2.012268,2.002232,2.136700,2.026404,2.022355,2.075626
2024-06-01,2.081474,2.098193,2.155615,2.096095,2.054590,2.119280,1.989884,2.155615,2.096095,1.985908,...,2.188193,2.028053,2.304990,2.183821,2.104497,1.938814,2.190383,2.028053,2.034146,2.129903
2024-06-02,1.966310,2.058873,2.326020,2.149336,2.040426,2.106776,2.132210,2.130078,2.044511,2.087900,...,2.181819,1.966310,2.519748,2.309795,2.234816,2.018105,2.250514,2.092080,2.016088,2.132210
2024-06-03,1.937473,1.980570,2.197641,2.237557,2.084199,2.088372,2.026651,2.156280,1.988508,2.124177,...,2.251023,1.835624,2.433632,2.287329,2.080035,2.055224,2.298794,2.186680,2.077956,2.034774
2024-06-04,1.872898,1.899303,2.495495,2.329112,2.080248,2.154346,2.012720,2.315179,1.918391,2.053380,...,2.354874,1.801261,2.533209,2.217736,1.970894,2.074017,2.215520,2.135044,2.055434,2.043138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-27,1.283963,3.507667,5.253798,1.587168,4.344676,5.737085,5.561964,3.560679,2.119015,3.380255,...,1.104012,1.999608,3.683824,6.746010,2.029828,2.742723,2.814969,11.178041,2.044087,2.840418
2025-06-28,1.268061,3.551922,5.272416,1.583264,4.325331,5.728701,5.592849,3.620053,2.145749,3.460762,...,1.107925,2.024836,3.649130,6.702555,2.055438,2.808046,2.766239,11.206441,2.055438,2.861909
2025-06-29,1.295714,3.501047,5.275439,1.585757,4.332141,5.772250,5.641003,3.604063,2.140548,3.511565,...,1.115231,1.997831,3.676870,6.713107,2.104466,2.837893,2.737546,11.201659,2.064859,2.877903
2025-06-30,1.290810,3.561815,5.146260,1.605235,4.367847,5.733180,5.619655,3.601211,2.102801,3.477349,...,1.107683,1.970466,3.692377,6.721224,2.065289,2.813053,2.743598,11.293985,2.057045,2.878503


We need to compute correlations and volatilities, we will use the average sample variance and correlations from the simulations

In [6]:
# Volatilities and correlations are computed on the relative daily returns
power_returns_df = power_prices_df.pct_change().dropna()
gas_returns_df = gas_prices_df.pct_change().dropna()

In [7]:
power_vol = power_returns_df.std(axis=0).mean()
gas_vol = gas_returns_df.std(axis=0).mean()
power_gas_corr = 0

for i in range(nb_scenarios):
    power_gas_corr += np.corrcoef(power_returns_df[i], gas_returns_df[i])[0,1]
    
# Divide to get the average
power_gas_corr /= nb_scenarios

Now we set the coefficients of the HRCO to make it fit the Two-Asset spread option framework. For this, recall that the payoff of our HRCO follows:

$$
P =  \max \bigr\{DV \times S_e - e^{-r \delta_t} \times[DV \times \{HR \times (S_g + GA) + VOM\} + SC \times (S_g + GA)], 0 \bigr\}
$$

Where
- $DV$ is the daily volume (in MWh),
- $S_e$ is the 1-day ahead (spot) electricity price for 24 hours of delivery (in $/MWh),
- $S_g$ is the 1-day ahead (spot) gas price for 24 hours of delivery (in $/MMBtu),
- $\delta_t$ is the time difference between the Energy Payment date and the delivery date (in years), 
- $HR$ is the Heat Ratio (in MMBTu/MWh),
- $GA$ is the gas adder (in $/MMBtu),
- $VOM$ is the Variable Operational and Maintenance costs (in $/MWh),
- $SC$ is the Start Cost (in MMBtu).

Linking to HRCO terminology, we have that

- $(HR \times (S_g + GA) + VOM)$ is the Energy Payment,
- $e^{-r \delta_t} \times (HR \times (S_g + GA) + VOM)$ is the Energy Payment adjusted by the fact that the payment is made $\delta_t$ days after delivery,
- $S_e - e^{-r \delta_t} \times (HR \times (S_g + GA) + VOM)$ is the Spark Spread.

Moreover, a general payoff profile for a Two-Asset Spread Option is of the form:

$$
P = \max \bigr[\alpha \times S_e - \beta \times S_g - K, 0 \bigr]
$$

And therefore, linking both, we have:

$$
\alpha = DV, \\
\beta = e^{-r \delta_t} \times (DV \times HR + SC), \\ 
K = e^{-r \delta_t} \times [DV \times (HR \times GA + VOM) + SC \times GA]
$$

In [13]:
option_prices_lists = []
spark_spread_lists = []

# We will compute one option for each expiry date that we have
for i, date in enumerate(simulation_dates):
    option_prices_list = []
    spark_spread_list = []
    
    # Compute delta_t in years (time difference between the Energy Payment date and delivery), taking into account the 1-day delivery delay
    if payment_grouped_monthly:
        days_to_end_month = ((date + pd.offsets.MonthEnd(0)) - date).days
        delta_t = (days_to_end_month + energy_payment_delay - 1) / 365.25
    else:
        delta_t = (energy_payment_delay - 1) / 365.25
        
    # Compute time to expiry of the current option (in years), taking into account the 1-day delivery delay
    time_to_expiry = ((date - start_simulation_date).days + 1) / 365.25
    
    for j in range(nb_scenarios):
        # Load the power and gas prices
        power_price = power_prices_df.iloc[i, j]
        gas_price = gas_prices_df.iloc[i, j]
        
        # Compute the coefficients for the HRCO option
        # Note that, since delta_t is not constant but rather depends on the day of the month, these coefficients need to be dinamically computed for each option.
        alpha = daily_volume
        beta = np.exp(-interest_rate * delta_t) * (daily_volume * heat_rate + start_charge)
        strike = np.exp(-interest_rate * delta_t) * (daily_volume * (heat_rate * gas_adder + VOM) + start_charge * gas_adder)
        
        # Add the spark spread to the list
        spark_spread_list.append(alpha * power_price - beta * gas_price - strike)

        # Create the option instance
        opt = SpreadOptionTwoAssets(
            type = "Call",
            underlying1 = power_price,
            underlying2 = gas_price,
            coef1 = alpha,
            coef2 = beta,
            strike = strike,
            time_to_expiry = time_to_expiry,
            volatility1 = power_vol, 
            volatility2 = gas_vol,
            correlation = power_gas_corr,
            rate = interest_rate
        )
         
        option_prices_list.append(opt.price())
    spark_spread_lists.append(spark_spread_list)
    option_prices_lists.append(option_prices_list)

In [9]:
spark_spread_df = pd.DataFrame(spark_spread_lists, index=simulation_dates, columns=range(nb_scenarios))
option_prices_df = pd.DataFrame(option_prices_lists, index=simulation_dates, columns=range(nb_scenarios))
payoff_df = spark_spread_df.copy().clip(lower=0)
time_value_df = option_prices_df.copy() - payoff_df.copy()

## WHY TIME VALUE < 0?

In [10]:
time_value_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
2024-05-31,-7.055995,-10.528318,-1.017348e+01,-14.464738,-8.780966,-10.878336,-8.804241,-12.960917,-6.792643,-8.925257,...,-12.867665,-7.821138,-11.257768,-14.474525,-11.996583,-15.639590,-14.460896,-7.729080e+00,-18.359137,-5.421878
2024-06-01,-4.857450,-17.595405,-2.141781e+00,-19.069180,-4.435675,-23.308915,-7.572894,-23.572463,3.920425,-7.020037,...,-20.799219,-12.094029,-11.244185,-18.973657,-23.454909,-26.011431,-24.309398,-1.164532e+01,-29.492102,-12.650859
2024-06-02,0.000005,-18.479788,4.014200e+01,-44.990273,-7.321492,-10.833475,-21.862620,-29.362775,-7.596781,-13.551318,...,-26.907698,-31.195489,-21.552042,-37.140647,-33.150679,-24.306562,-69.325465,-1.706240e+01,-57.239463,-23.775240
2024-06-03,-17.361289,-33.472385,3.877483e-12,-101.385537,-40.345110,-11.138870,-31.690314,-41.642445,-14.686073,-47.139784,...,-38.265602,-49.474591,-18.063242,-70.423364,-35.974664,-30.472918,-124.322151,-2.438016e+01,-92.104311,-30.846794
2024-06-04,-11.609171,-31.565052,1.912640e+02,-149.253740,-40.190975,41.440299,-65.141582,-57.147852,-13.779053,-82.158963,...,-53.600195,-52.260296,-35.049504,-86.550287,-46.188063,-36.913161,-140.246348,-1.642074e+01,-147.404420,-20.729206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-27,-6705.319635,-4857.900434,1.107380e-03,-17913.696057,14545.289141,823.677299,9761.248908,4940.671835,1772.161982,3364.615651,...,-5512.424808,392.587938,1470.269112,-27196.960563,7762.000596,-12723.542516,-13953.887291,3.833401e-02,-8126.798579,-3145.399076
2025-06-28,-3527.071772,9802.928978,2.944300e-09,-11088.852054,663.898593,1.375090,69.951981,25.002636,1467.549258,2047.274304,...,-2834.118995,0.786424,3.415353,-13313.473617,156.642701,-6160.618988,-7027.010985,1.576142e-07,-3871.036318,11465.836817
2025-06-29,-3611.890095,6662.577213,1.877868e-09,-11512.466143,828.233718,1.652833,65.513223,17.708611,1531.901438,2001.744733,...,-2764.443144,1.195565,3.734165,-13525.922525,127.511902,-6500.494490,-7186.156697,2.078454e-07,-4040.007928,10568.065794
2025-06-30,-4861.993203,-851.712415,2.412303e-06,-14570.125009,5149.685186,60.768060,1285.761710,224.475341,9052.510034,11370.551751,...,-3848.580020,41.998200,106.941128,-19672.328833,1517.791166,-9214.754987,-10138.819974,3.954019e-05,-5798.171507,2321.255062
